In [ ]:
import boto3
import os
import sys
import csv
import pandas as pd
import imghdr

In [ ]:
## AWS client 
client=boto3.client('rekognition')

### Only Run this Section Once

In [ ]:
## Run recursive API call to AWS Rekognition for image classification
main_dir = 'C:/Users/dakin/Documents/Development/ResonanceCompanies/imageSet/'
product_list = []
    
for root, directories, filenames in os.walk(main_dir):
    for filename in filenames:
        
        ## create product_id and select image file to examine
        product_id = os.path.join(os.path.basename(root), filename)
        imageFile = (os.path.join(root,filename))
        if imghdr.what(imageFile) not in ['jpeg', 'png']:
            continue
        else:
            ## open image and run label detection
            with open(imageFile, 'rb') as image:
                response = client.detect_labels(Image={'Bytes': image.read()}) #, MinConfidence=95

            ## add label to dataframe as attribute
            for label in response['Labels']:
                product_list.append({'product_id' : product_id, 
                                     'label' : label['Name'] , 
                                     'confidence' : label['Confidence']})     

In [ ]:
## Create Dataframe from list and export to CSV to be used in the next step
product_meta = pd.DataFrame(product_list)
product_meta.to_csv('product_meta.csv', encoding='utf-8')